# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [63]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'nba_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [4]:
# Read Data
dataset = Dataset.get_by_name(ws, name='nba-games-data')
df = dataset.to_pandas_dataframe()

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.15)

In [30]:
x_train = train_data.iloc[:,:-1]
y_train = train_data['HOME_TEAM_WINS']
x_test = test_data.iloc[:,:-1]
y_test = test_data['HOME_TEAM_WINS']
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

x_train.to_csv('./data/x_train.csv', index=False)
y_train.to_csv('./data/y_train.csv', index=False)
x_test.to_csv('./data/x_test.csv', index=False)
y_test.to_csv('./data/y_test.csv', index=False)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = 'nba-compute'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_V2', max_nodes=5)
train_cluster = ComputeTarget.create(ws, compute_name, compute_config)
train_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [33]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1),
        '--max_iter': choice(10, 20, 50, 100, 250)
    }
)

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(
    name='sklearn_env',
    file_path='conda_dependencies.yml'
)

estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    arguments=['--data_directory', './data'],
    environment=sklearn_env,
    compute_target=train_cluster
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    policy=early_termination_policy,
    hyperparameter_sampling=param_sampling,
    primary_metric_name='AUC_weighted',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=5,
    max_concurrent_runs=5
)

In [50]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [51]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [54]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_metrics()

{'AUC_weighted': 0.9250828222408249}

In [65]:
best_run.get_details()

{'runId': 'HD_541f3743-ad36-4315-9651-ce18a3b2182a_0',
 'target': 'nba-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-11-15T17:36:50.019346Z',
 'endTimeUtc': '2022-11-15T17:37:04.560615Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '1ec8bca2-1ef0-4ec8-bd53-5ef01e695875',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data_directory',
   './data',
   '--C',
   '0.01',
   '--max_iter',
   '100'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'nba-compute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credent

In [55]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [121]:
best_model = best_run.register_model(
    model_name='nba-hyperdrive', 
    model_path='outputs/model.joblib',
    tags={"version":"2"})

Model(workspace=Workspace.create(name='quick-starts-ws-215571', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-215571'), name=nba-hyperdrive, id=nba-hyperdrive:2, version=2, tags={'version': '2'}, properties={})

In [129]:
from azureml.core.model import Model

In [133]:
best_model = best_run.register_model(
    model_name='nba-hyperdrive', 
    model_path='outputs/model.joblib',
    tags={"version":"2"}
)

In [142]:
print(best_model.name, best_model.version)

nba-hyperdrive 4


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [134]:
sklearn_env.register(ws)

{
    "assetId": "azureml://locations/southcentralus/workspaces/c6aa564e-2b91-4e62-89b9-4a59fcc42ab7/environments/sklearn_env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "AZUREML_ENTRY_SCRIPT": "odl_user_215571/hyperparameter_deploy.py",
        "AZUREML_SOURCE_DIRECTORY": "odl_u

In [169]:
from azureml.core.model import InferenceConfig
hyperparameter_inf_config = InferenceConfig(
    environment=ws.environments['sklearn_env'],
    source_directory='.',
    entry_script='hyperparameter_deploy.py'
)

In [170]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, enable_app_insights=True)

In [171]:
from azureml.core.model import Model
service = Model.deploy(
    ws, 
    "hyperdrive-service",
    [best_model],
    hyperparameter_inf_config,
    deployment_config
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-15 22:48:25+00:00 Creating Container Registry if not exists.
2022-11-15 22:48:25+00:00 Registering the environment.
2022-11-15 22:48:26+00:00 Use the existing image.
2022-11-15 22:48:26+00:00 Generating deployment configuration.
2022-11-15 22:48:27+00:00 Submitting deployment to compute.
2022-11-15 22:48:30+00:00 Checking the status of deployment hyperdrive-service..
2022-11-15 22:51:45+00:00 Checking the status of inference endpoint hyperdrive-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [172]:
sample_data = x_test[:1].to_json()
headers = {'Content_Type': 'application/json'}

In [173]:
import requests
import json
uri = service.scoring_uri
resp = requests.post(uri, sample_data, headers)

In [176]:
resp

<Response [502]>

In [175]:
print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [114]:
service.get_logs()

In [168]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

